# Non Periodic Issues

The goal of this jupyter notebook is to find a good example of what I think is occuring. 

i.e. When we break grants a lot then we keep being pushed into selecting the same resource as others select which is a bad thing.

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from matplotlib.ticker import FormatStrFormatter

In [ ]:
import json
import os
import random

In [ ]:
# JSON file containing the results for this simulation run
results_file = "/Users/brianmccarthy/git_repos/results-analysis/configs/cv2x.json"

In [ ]:
figure_folder = "../data/figures/"

In [ ]:
with open(results_file) as results_json:
    config = json.load(results_json)

In [ ]:
# Markers to use for this run
markers = [".", "o", "v", "^", "<", ">", "1", "2", "3", "4", "8", "s", "p", "P", "*", "h",
           "H", "+", "x", "X", "D", "d", "|", "_", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [ ]:
def create_bins(lower_bound, width, quantity):
    """ create_bins returns an equal-width (distance) partitioning.
        It returns an ascending list of tuples, representing the intervals.
        A tuple bins[i], i.e. (bins[i][0], bins[i][1])  with i > 0
        and i < quantity, satisfies the following conditions:
            (1) bins[i][0] + width == bins[i][1]
            (2) bins[i-1][0] + width == bins[i][0] and
                bins[i-1][1] + width == bins[i][1]
    """
    bins = []
    for low in range(lower_bound, lower_bound + quantity * width + 1, width):
        bins.append((low, low + width))
    return bins

In [ ]:
def bin_fields(df, fields, bin_width=10, bin_quantity=49):
    """
    Bins multiple dfs into a single dictionary that can be used as an average for multiple fields across multiple
    runs
    :param df: dataframe to bin
    :param fields: fields to be binned.
    :param bin_width: width of each bin
    :param bin_quantity: total number of bins
    :return:
    """
    bins = create_bins(lower_bound=0, width=bin_width, quantity=bin_quantity)
    distances = []
    overall_fields = {}
    for interval in bins:
        upper_b = interval[1]
        distances.append(upper_b)

    for field in fields:
        print("{} being binned".format(field))
        overall_fields[field] = []

    overall_fields["distance"] = distances

    distance_col = config["results"]["distance"]

    for i in range(len(bins)):
        lower_b = bins[i][0]
        upper_b = bins[i][1]
        fields_temp = df[(df[distance_col] >= lower_b) & (df[distance_col] < upper_b)]
        for field in fields:
            if i < len(overall_fields[field]):
                overall_fields[field][i] = (fields_temp[field].mean() + overall_fields[field][i]) / 2
            else:
                overall_fields[field].append(fields_temp[field].mean())

    return overall_fields

In [ ]:
def pdr_dist_individual(pdr, distances, label, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, pdr, label=label, marker=markers[0], markevery=3)

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower right')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}/{}.png".format(figure_folder, plot_name), dpi=300)

In [ ]:
def pdr_dist(pdrs, distances, labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(pdrs)):
        ax.plot(distances, pdrs[i], label=labels[i], marker=markers[i], markevery=3)

    ax.set(xlabel='Distance (m)', ylabel='Packet Delivery Rate (PDR) %')
    ax.legend(loc='lower right')
    ax.tick_params(direction='in')
    
    ax.set_ylim([0, 100])
    plt.yticks(np.arange(0, 101, step=10))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}/{}.png".format(figure_folder, plot_name), dpi=300)

In [ ]:
def errors_dist_individual(distances, decoded, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    ax.plot(distances, decoded, label="Decoded", marker=self.markers[0], markevery=3)

    for i in range(len(errors)):
        ax.plot(distances, errors[i], label=error_labels[i], marker=self.markers[i+1], markevery=3)

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))

    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()

    if store:
        fig.savefig("{}/{}.png".format(figure_folder, plot_name), dpi=300)

In [ ]:
def errors_dist(distances, decoded, decoded_labels, errors, error_labels, plot_name, show=True, store=False):
    fig, ax = plt.subplots()

    for i in range(len(decoded)):
        ax.plot(distances, decoded[i], label=decoded_labels[i], marker=self.markers[i], markevery=3)

        for j in range(len(errors[i])):
            ax.plot(distances, errors[i][j], label=error_labels[i][j], marker=self.markers[i + j])

    ax.legend(loc='center left')
    ax.set(xlabel='Distance (m)', ylabel='Messages Decoded')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=.1))

    ax.set_xlim([0, (max(distances) + 1)])
    plt.xticks(np.arange(0, (max(distances) + 1), step=50))
    
    fig.suptitle(plot_name, fontsize=12)
    
    if show:
        fig.show()
        
    if store:
        fig.savefig("{}/{}.png".format(figure_folder, plot_name), dpi=300)

### CBR reporting

In [ ]:
raw_data_folder = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/DCC-Enabled"

In [ ]:
first_file = True
merged_df = pd.DataFrame()
for file in os.listdir(raw_data_folder):
    if ".csv" in file:
        result_file = raw_data_folder + "/" + file
        if first_file:
            merged_df = pd.read_csv(result_file)
            first_file = False
        else:
            merged_df.append(pd.read_csv(result_file))

In [ ]:
merged_df.plot(x="Time", y="cbr", kind="line")

Need to do some more checks, in much better shape than it was but still some issues here.

- Primarily need to see why we have such a large fluctuation
- Also have to look at why it is a little low, though it also is an average of all vehicles.
- Might find min and max guys

In [ ]:
node_23_df = merged_df[(merged_df["NodeID"] == 23) & (merged_df["cbr"] >= 0.0)]

In [ ]:
node_23_df.plot(x="Time", y="cbr", kind="line")

Wild fluctuation is a major issue, need to investigate what the story is there

Also extremely noisy, reducing the number of CBR reports is necessary, possibly sending it every 10ms or something like that will improve the quality. 100ms too wide a band.

In [ ]:
reduced_merged_df = merged_df[(merged_df["Time"] >= 510) & (merged_df["Time"] <= 511)]

In [ ]:
reduced_merged_df.plot(x="Time", y="cbr", kind="line")

In [ ]:
print("mean: {}".format(reduced_merged_df["cbr"].mean()))
print("std: {}".format(reduced_merged_df["cbr"].std()))
print("median: {}".format(reduced_merged_df["cbr"].median()))

In [ ]:
fig, ax = plt.subplots()

ax.plot(node_23_df["Time"], node_23_df["cbr"], label="CBR")

ax.set(xlabel='Time (s)', ylabel='Channel Busy Ratio %')
ax.legend(loc='lower left')
ax.tick_params(direction='in')

ax.set_ylim([0, 0.60])
plt.yticks(np.arange(0, 0.61, step=0.05))

ax.set_xlim([min(node_23_df["Time"] + 2), (max(node_23_df["Time"]))])
plt.xticks(np.arange(min(node_23_df["Time"] + 2), (max(node_23_df["Time"]) + 1), step=1))

fig.suptitle("CBR over time", fontsize=12)
plt.savefig("CBR_sci_info_record.png", dpi=300)

### Channel Usage

In [ ]:
merged_df = merged_df[merged_df["Time"] >= 502]

In [ ]:
node_23_df = merged_df[(merged_df["NodeID"] == 23)]

In [ ]:
# node_23_df = merged_df

In [ ]:
node_23_df["subchannelReceived"] = np.where(node_23_df["sciFailedHalfDuplex"].eq(1.0), -1.0, node_23_df["subchannelReceived"])


In [ ]:
node_23_df_reduced = node_23_df[(node_23_df["Time"] >= 511.0) & (node_23_df["Time"] <= 511.2)]

In [ ]:
small_df = node_23_df_reduced

In [ ]:
small_df = small_df.round({'Time': 3})

In [ ]:
small_df["subchannelReceived"] = small_df["subchannelReceived"].fillna(-2)

In [ ]:
group_df = small_df.groupby("Time")["subchannelReceived"].value_counts().unstack().fillna(0)

In [ ]:
group_df.loc[group_df[-1.0] == 1, [0.0, 1.0, 2.0]] = -1

In [ ]:
group_df = group_df[[0.0, 1.0, 2.0]]

In [ ]:
group_df.head()

In [ ]:
group_df = group_df.T

In [ ]:
group_df.index

In [ ]:
group_df.columns

In [ ]:
group_df = group_df.astype("int")

In [ ]:
len(group_df.values[0])

In [ ]:
np_xticks = np.arange(min(group_df.columns), max(group_df.columns), 0.01)

In [ ]:
np_xticks = np.round(np_xticks, 2)

In [ ]:
subframes = list(np.arange(len(group_df.columns), step=20))

In [ ]:
fig, ax = plt.subplots(figsize=(100,30))
im = ax.imshow(group_df.values, interpolation="nearest")

# We want to show all ticks...
ax.set_xticks(np.arange(len(group_df.columns), step=20))
ax.set_yticks(np.arange(len(group_df.index)))

# ax.set_xticklabels(np_xticks)
ax.set_xticklabels(np.array(group_df.columns)[subframes])

for i in range(len(group_df.values)):
    for j in range(len(group_df.values[i])):
        if group_df.values[i,j] == -1:
            text = ax.text(j, i, "HD", ha="center", va="center", color="red")
        elif group_df.values[i,j] == 0:
            text= ax.text(j, i, 0, ha="center", va="center", color="Orange")
        else:
            text = ax.text(j, i, group_df.values[i,j], ha="center", va="center", color="white")

# fig.tight_layout()
plt.gcf().set_facecolor("black")
# fig.set_size_inches(600, 100, forward=True)
plt.savefig("{}/Message_history.svg".format(figure_folder))

In [ ]:
merged_df.columns

In [ ]:
merged_df.head()

In [ ]:
grant_request_df = merged_df[(merged_df["grantStartTime"] > 0) | (merged_df["grantBreakMissedTrans"])][["Time", "NodeID", "grantStartTime", "selectedSubchannelIndex", "grantBreakMissedTrans", "grantBreakSize"]]

In [ ]:
grant_request_df["grantStartTime"] = grant_request_df["grantStartTime"].round(3)

In [ ]:
grant_request_df.shape

In [ ]:
group = grant_request_df.sort_values("grantStartTime")

In [ ]:
duplicateRowsDF = grant_request_df[grant_request_df.duplicated(['grantStartTime', "selectedSubchannelIndex"], keep=False)]

In [ ]:
duplicateRowsDF = duplicateRowsDF.sort_values("grantStartTime")

In [ ]:
duplicateRowsDF.count()

In [ ]:
grant_request_df.shape

In [ ]:
duplicateRowsDF.shape

In [ ]:
group_df[group_df==1].count().sum()

In [ ]:
merged_df[merged_df["tbSent"] > 0]["tbSent"].sum()

In [ ]:
(23698/10)/10

In [ ]:
(236.98/300) * 100

In [ ]:
temp_df = grant_request_df

In [ ]:
# temp_df["grantStartTime"] = temp_df["grantStartTime"].astype(str)

In [ ]:
temp_df.head()

In [ ]:
duplicateRowsDF = grant_request_df[grant_request_df.duplicated(['grantStartTime', "selectedSubchannelIndex"], keep=False)]

In [ ]:
duplicateRowsDF = duplicateRowsDF.sort_values(["grantStartTime", "selectedSubchannelIndex", "Time"])

## Random selection

In [ ]:
def random_selection(raw_data_folder):    
    files = os.listdir(raw_data_folder)
    count = 0
    file = ""
    while ".csv" not in file and count < len(files) * 10:
        file = random.choice(files)
        count += 1

    result_file = raw_data_folder + "/" + file
    print("Random result file selected: {} in {} selections".format(result_file, count))
    random_df = pd.read_csv(result_file)

    # Reduce to the time we want which is from 502 onwards.
    random_df = random_df[random_df["Time"] >= 502]

    node = np.random.choice(random_df["NodeID"].unique(), 1)[0]
#     print("Node: {}".format(node))
    
    return random_df, node

## CBR individual

In [ ]:
def plot_cbr(dfs, labels, plot_name):

    fig, ax = plt.subplots()

    for i in range(len(dfs)):
        df = dfs[i]
        ax.plot(df["Time"], df["cbr"], label=labels[i])

    ax.set(xlabel='Time (s)', ylabel='Channel Busy Ratio %')
    ax.legend(loc='lower left')
    ax.tick_params(direction='in')

    ax.set_ylim([0, 1])
    plt.yticks(np.arange(0, 1.1, step=0.1))

    ax.set_xlim([min(df["Time"]), (max(df["Time"]))])
    plt.xticks(np.arange(min(df["Time"]), (max(df["Time"]) + 1), step=1))

    plt.savefig("{}/{}.png".format(figure_folder, plot_name), dpi=300)

In [ ]:
folder_path = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/"
interested_folders = ["NO-CC", "DCC-Enabled", "After-1", "After-5", "Highway-fast", "NO-CC-high-density", "After-1-high-density", "After-5-high-density"]
dfs = {}
selected_node = False
for folder in os.listdir(folder_path):
    if folder in interested_folders:
        result_path = folder_path + folder
        random_df, node = random_selection(result_path)
        if not selected_node:
            selected_node = node
            print("Selected Node: {}".format(selected_node))
        random_df = random_df[(random_df["NodeID"] == selected_node) & (random_df["cbr"].notnull())]
        dfs[folder] = random_df[["Time", "cbr"]]
        

In [ ]:
motivational_cbr = []
motivational_cbr.append(dfs["Highway-fast"])
motivational_cbr.append(dfs["NO-CC"])
motivational_cbr.append(dfs["NO-CC-high-density"])

cbr_medium = []
cbr_medium.append(dfs["DCC-Enabled"])
cbr_medium.append(dfs["NO-CC"])
cbr_medium.append(dfs["After-1"])
cbr_medium.append(dfs["After-5"])

cbr_high = []
cbr_high.append(dfs["DCC-Enabled"])
cbr_high.append(dfs["NO-CC-high-density"])
cbr_high.append(dfs["After-1-high-density"])
cbr_high.append(dfs["After-5-high-density"])

In [ ]:
plot_cbr(motivational_cbr, ["Highway Fast", "Medium Density", "High Density"], "motivationalCBR")

In [ ]:
plot_cbr(cbr_medium, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "medium-CBR")

In [ ]:
plot_cbr(cbr_high, ["DCC Access", "No Congestion Control", "3GPP DCC Mechanism (CBR) - Immediate new grant", "3GPP DCC Mechanism (CBR) - 5 missed trans b4 new grant"], "high-CBR")

## Collisions issue

Section includes code to generate the collision causes and what happens when a grant is generated.

In [ ]:
raw_data_folder = "/Users/brianmccarthy/git_repos/results-analysis/data/raw_data/cv2x/DCC-Enabled"

In [ ]:
def random_selection_grant(raw_data_folder):    
    files = os.listdir(raw_data_folder)
    count = 0
    file = ""
    while ".csv" not in file and count < len(files) * 10:
        file = random.choice(files)
        count += 1

    result_file = raw_data_folder + "/" + file
    print("Random result file selected: {} in {} selections".format(result_file, count))
    random_df = pd.read_csv(result_file)

    # Reduce to the time we want which is from 502 onwards.
    random_df = random_df[random_df["Time"] >= 502]

    node = np.random.choice(random_df["NodeID"].unique(), 1)[0]
    print("Node: {}".format(node))
    subsection_df = random_df[random_df["NodeID"] == node]
    grant_time = np.random.choice(subsection_df["grantStartTime"].unique(), 1)[0]
    selectedSubchannel = subsection_df[subsection_df["grantStartTime"] == grant_time]["selectedSubchannelIndex"]
    selectedSubchannel = selectedSubchannel.get_values()[0]
    print("Grant Time: {}, subchannel: {}".format(grant_time, selectedSubchannel))
    generated_time = subsection_df[subsection_df["grantStartTime"] == grant_time]["Time"]
    generated_time = generated_time.get_values()[0]
    print("Time of generation: {}".format(generated_time))
    history_time = generated_time - 0.1
    print("Minimum history time: {}".format(history_time))
    future_time = generated_time + 0.1
    print("Maximum Future grant time: {}".format(future_time))
    
    return random_df, node, generated_time, grant_time, selectedSubchannel, history_time, future_time

In [ ]:
df, node, generated_time, grant_time, subchannel, history_time, future_time = random_selection_grant(raw_data_folder)

In [ ]:
df["grantBreakMissedTrans"].count()

In [ ]:
df["grantBreak"].count()

In [ ]:
df["grantBreakSize"].count()

In [ ]:
df["grantStartTime"].count()

In [ ]:
subframeIndex = int((grant_time - generated_time)* 1000)

In [ ]:
df = df[(df["Time"] >= history_time) & (df["Time"] <= future_time)]

In [ ]:
df_removed = df.drop(["generatedGrants", "grantBreak", "tbSent", "tbFailedDueToProp", "tbFailedDueToInterference", "missedTransmission", "grantStartTime", "grantBreakSize", "grantBreakMissedTrans", "sciFailedDueToInterference", "sciFailedDueToProp", "subchannelSent", "subchannelsUsedToSend", "selectedSubchannelIndex", "selectedNumSubchannels", "subchannelsUsed", "sciSent"], axis=1)

In [ ]:
df_removed = df_removed[df_removed["sciDecoded"].notnull()]

In [ ]:
for column in ["tbDecoded", "tbFailedButSCIReceived", "tbFailedDueToNoSCI", "tbFailedHalfDuplex", "tbReceived", "txRxDistanceTB"]:
    df_removed[column].fillna(-1, inplace=True)

In [ ]:
tb_df = df_removed.drop(["cbr", "interPacketDelay", "posX", "posY", "senderID","sciDecoded","sciFailedHalfDuplex","sciNotDecoded","sciReceived","subchannelReceived","txRxDistanceSCI"], axis=1)
sci_df = df_removed.drop(["tbDecoded","tbFailedButSCIReceived","tbFailedDueToNoSCI","tbFailedHalfDuplex","tbReceived","txRxDistanceTB"], axis=1)

scis_with_tb = sci_df[sci_df["txRxDistanceSCI"].isin(tb_df["txRxDistanceTB"].unique())]
tbs_there = tb_df[tb_df["tbDecoded"] != -1]

scis_without_tb = sci_df[~sci_df["txRxDistanceSCI"].isin(tb_df["txRxDistanceTB"].unique())]
tbs_missing = tb_df[tb_df["tbDecoded"] == -1]

scis_with_tb = scis_with_tb.merge(tbs_there, left_on=["NodeID", "Time", "txRxDistanceSCI"], right_on=["NodeID", "Time", "txRxDistanceTB"])

scis_without_tb_merge = scis_without_tb.merge(tbs_missing, how="right", left_on=["NodeID", "Time"], right_on=["NodeID", "Time"])
scis_without_tb_merge = scis_without_tb_merge.drop_duplicates(subset="txRxDistanceSCI")

df_removed = scis_without_tb_merge.append(scis_with_tb, sort=False)

In [ ]:
subchannels = 3
before_grant = []
after_grant = []
for i in range(subchannels):
    before_grant.append([])
    after_grant.append([])
    
before_grant_df = df_removed[df_removed["Time"] <= generated_time]
after_grant_df = df_removed[df_removed["Time"] >= generated_time]

# 0 = F
# 1 = T
time = round(history_time, 3)
index = 0
while time < generated_time:
    # Fill in this subframe
    for channel in before_grant:
        channel.append(0)
    filtered_df = before_grant_df[(before_grant_df["Time"] == time) & (before_grant_df["NodeID"] == node) & (before_grant_df["sciDecoded"] == 1)]
    if not filtered_df.empty:
        subchs = filtered_df["subchannelReceived"].values
        tbDecods = filtered_df["tbDecoded"].values
        for i in range(len(tbDecods)):
            if tbDecods[i] == 1:
                before_grant[int(subchs[i])][index] = 1
            elif tbDecods[i] == -1:
                before_grant[int(subchs[i][index])] = 2
    time = round(time + 0.001, 3)
    index += 1

# 0 = F -> F
# 1 = T -> F
# 2 = F -> T
# 3 = T -> T
# 4 = F -> M
# 5 = T -> M
# 6 = Selected
time = round(generated_time, 3)
index = 0
while time < future_time:
    # Fill in this subframe
    for i in range(len(after_grant)):
        if before_grant[i][index] == 0:
            after_grant[i].append(0)
        else:
            after_grant[i].append(1)
    filtered_df = after_grant_df[(after_grant_df["Time"] == time) & (after_grant_df["NodeID"] == node)]
    if not filtered_df.empty:
        subchs = filtered_df["subchannelReceived"].values
        tbDecods = filtered_df["tbDecoded"].values
        for i in range(len(tbDecods)):
            if tbDecods[i] == 1 and before_grant[int(subchs[i])][index] == 0:
                after_grant[int(subchs[i])][index] = 2
            elif tbDecods[i] == 1 and before_grant[int(subchs[i])][index] == 1:
                after_grant[int(subchs[i])][index] = 3
            elif tbDecods[i] == 1 and before_grant[int(subchs[i])][index] == 2:
                after_grant[int(subchs[i])][index] = 4
            elif tbDecods[i] == -1 and before_grant[int(subchs[i])][index] == 0:
                after_grant[int(subchs[i])][index] = 5
            elif tbDecods[i] == -1 and before_grant[int(subchs[i])][index] == 1:
                after_grant[int(subchs[i])][index] = 6
            elif tbDecods[i] == -1 and before_grant[int(subchs[i])][index] == 2:
                after_grant[int(subchs[i])][index] = 7
                
    if index == subframeIndex:
        after_grant[int(subchannel)][index] = 6
    time = round(time + 0.001, 3)
    index += 1

In [ ]:
after_grant_df[after_grant_df["NodeID"] == 220]["tbDecoded"].unique()

In [ ]:
print(len(before_grant), len(before_grant[0]))
print(len(after_grant), len(after_grant[0]))

In [ ]:
fig, ax = plt.subplots(figsize=(30, 3))
im = ax.imshow(before_grant, interpolation="nearest", cmap=plt.cm.Paired_r)

# We want to show all ticks...
ax.set_xticks(np.arange(len(before_grant[0]), step=20))
ax.set_yticks(np.arange(len(before_grant)))

# ax.set_xticklabels(np_xticks)
# ax.set_xticklabels(np.array(before_grant)[subframes])

for i in range(len(before_grant)):
    for j in range(len(before_grant[i])):
        if before_grant[i][j] == 1:
            text = ax.text(j, i, "T", ha="center", va="center", color="Black")
        elif before_grant[i][j] == -1:
            text = ax.text(j, i, "M", ha="center", va="center", color="Black")
        else:
            text = ax.text(j, i, "F", ha="center", va="center", color="Black")

plt.gcf().set_facecolor("black")
plt.savefig("{}/{}.png".format(figure_folder, "test_history"), dpi=300)

In [ ]:
# 0 =  F -> F  = F
# 1 =  T -> F  = R
# 2 =  M -> T  = MT
# 3 =  F -> T  = N
# 4 =  T -> T  = T
# 5 =  F -> M  = NM
# 6 =  T -> M  = M
# 7 =  M -> M  = MM
# 8 = Selected = S

fig, ax = plt.subplots(figsize=(30, 3))
im = ax.imshow(after_grant, interpolation="nearest", cmap=plt.cm.Paired_r)

# We want to show all ticks...
ax.set_xticks(np.arange(len(after_grant[0]), step=20))
ax.set_yticks(np.arange(len(after_grant)))

for i in range(len(after_grant)):
    for j in range(len(after_grant[i])):
        if after_grant[i][j] == 0:
            text = ax.text(j, i, "F", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 1:
            text= ax.text(j, i, "R", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 2:
            text= ax.text(j, i, "MT", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 3:
            text= ax.text(j, i, "N", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 4:
            text= ax.text(j, i, "N", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 5:
            text= ax.text(j, i, "T", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 6:
            text= ax.text(j, i, "NM", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 7:
            text= ax.text(j, i, "M", ha="center", va="center", color="Black")
        elif after_grant[i][j] == 8:
            text= ax.text(j, i, "MM", ha="center", va="center", color="Black")
        else:
            text = ax.text(j, i, "F", ha="center", va="center", color="Black")

plt.gcf().set_facecolor("black")
plt.savefig("{}/{}.png".format(figure_folder, "test_future"), dpi=300)